In [1]:
# https://www.kaggle.com/vkrahul/twitter-hate-speech

In [53]:
import pandas as pd
import os
import re
import spacy
from gensim.models.phrases import Phrases, Phraser
from time import time 
import multiprocessing
from gensim.models import Word2Vec
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import scale
import keras 
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, Input, Embedding
from keras.layers.merge import Concatenate
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics import confusion_matrix
import matplotlib as plt

In [33]:
df = pd.read_csv('train.csv')
del(df['id'])

In [35]:
df_clean = df
from nltk.tokenize import RegexpTokenizer
t = time()


tokenizer = RegexpTokenizer(r'\w+')
df_clean['clean'] = df_clean['tweet'].astype('str') 
df_clean.dtypes

df_clean["tokens"] = df_clean["clean"].apply(tokenizer.tokenize)
# delete Stop Words

print('Time to tokenize everything: {} mins'.format(round((time() - t) / 60, 2)))
df_clean.head()

Time to tokenize everything: 0.0 mins


,label,tweet,clean,tokens
0,0,@user when a father is dysfunctional and is s...,@user when a father is dysfunctional and is s...,"[user, when, a, father, is, dysfunctional, and..."
1,0,@user @user thanks for #lyft credit i can't us...,@user @user thanks for #lyft credit i can't us...,"[user, user, thanks, for, lyft, credit, i, can..."
2,0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]"
3,0,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...,"[model, i, love, u, take, with, u, all, the, t..."
4,0,factsguide: society now #motivation,factsguide: society now #motivation,"[factsguide, society, now, motivation]"


In [37]:
import gensim
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('/Users/0x23/Desktop/*/Projects/GoogleNews-vectors-negative300.bin', binary = True)


In [38]:
#WORD2VEC()
cores = multiprocessing.cpu_count() # Count the number of cores in a computer, important for a parameter of the model
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

#BUILD_VOCAB()
t = time()
w2v_model.build_vocab(df_clean["tokens"], progress_per=1000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

#TRAIN()
w2v_model.train(df_clean["tokens"], total_examples=w2v_model.corpus_count, epochs=10000, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.01 mins
Time to train the model: 36.08 mins


In [42]:
#words similar to movie
w2v_model.wv.most_similar(positive=["movie"])

#words similar to good
w2v_model.wv.most_similar(positive=["good"])

[('great', 0.27397453784942627),
 ('morning', 0.23787921667099),
 ('nice', 0.20033378899097443),
 ('wednesday', 0.19090279936790466),
 ('sad', 0.1897774487733841),
 ('worse', 0.1849452406167984),
 ('like', 0.18481267988681793),
 ('bad', 0.1712118536233902),
 ('bing', 0.16550961136817932),
 ('big', 0.1623874306678772)]

In [43]:
# defining the chart
output_notebook()
plot_tfidf = bp.figure(plot_width=700, plot_height=600, title="A map of 10000 word vectors",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)

# getting a list of word vectors. limit to 10000. each is of 200 dimensions
word_vectors = [w2v_model[w] for w in list(w2v_model.wv.vocab.keys())[:5000]]

# dimensionality reduction. converting the vectors to 2d vectors
tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
tsne_w2v = tsne_model.fit_transform(word_vectors)

# putting everything in a dataframe
tsne_df = pd.DataFrame(tsne_w2v, columns=['x', 'y'])
tsne_df['words'] = list(w2v_model.wv.vocab.keys())[:5000]

# plotting. the corresponding word appears when you hover on the data point.
plot_tfidf.scatter(x='x', y='y', source=tsne_df)
hover = plot_tfidf.select(dict(type=HoverTool))
hover.tooltips={"word": "@words"}
show(plot_tfidf)

Loading BokehJS ...

ipykernel_launcher:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 2043 samples in 0.023s...
[t-SNE] Computed neighbors for 2043 samples in 2.144s...
[t-SNE] Computed conditional probabilities for sample 1000 / 2043
[t-SNE] Computed conditional probabilities for sample 2000 / 2043
[t-SNE] Computed conditional probabilities for sample 2043 / 2043
[t-SNE] Mean sigma: 6.455959
[t-SNE] KL divergence after 250 iterations with early exaggeration: 156.704605
[t-SNE] KL divergence after 1000 iterations: 3.236742


In [44]:
#First defining the X (input), and the y (output)
y = df['label'].values
X = np.array(df_clean["tokens"])

#And here is the train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [45]:
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform([x for x in X_train])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print ('vocab size :', len(tfidf))

vocab size : 2964


In [49]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += w2v_model[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [50]:
train_vecs_w2v = np.concatenate([buildWordVector(z, 300) for z in map(lambda x: x, X_train)])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, 300) for z in map(lambda x: x, X_test)])
test_vecs_w2v = scale(test_vecs_w2v)

print ('shape for training set : ',train_vecs_w2v.shape,'\nshape for test set : ', test_vecs_w2v.shape)

ipykernel_launcher:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
ipykernel_launcher:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
shape for training set :  (25569, 300) 
shape for test set :  (6393, 300)


In [54]:
model = Sequential()

model.add(Dense(128, activation='relu', input_dim=300))
model.add(Dropout(0.7))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               38528     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 38,657
Trainable params: 38,657
Non-trainable params: 0
_________________________________________________________________


In [55]:
history = model.fit(train_vecs_w2v, y_train, epochs=20, batch_size=50,validation_data=(test_vecs_w2v,y_test))
loss, accuracy = model.evaluate(train_vecs_w2v, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(test_vecs_w2v, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/20
512/512 [==============================] - 1s 2ms/step - loss: 0.6768 - accuracy: 0.6890 - val_loss: 0.4326 - val_accuracy: 0.8573
Epoch 2/20
512/512 [==============================] - 1s 1ms/step - loss: 0.6614 - accuracy: 0.6973 - val_loss: 0.4238 - val_accuracy: 0.8641
Epoch 3/20
512/512 [==============================] - 1s 1ms/step - loss: 0.6463 - accuracy: 0.7100 - val_loss: 0.4154 - val_accuracy: 0.8692
Epoch 4/20
512/512 [==============================] - 1s 1ms/step - loss: 0.6419 - accuracy: 0.7064 - val_loss: 0.4074 - val_accuracy: 0.8730
Epoch 5/20
512/512 [==============================] - 1s 1ms/step - loss: 0.6244 - accuracy: 0.7185 - val_loss: 0.3999 - val_accuracy: 0.8772
Epoch 6/20
512/512 [==============================] - 1s 2ms/step - loss: 0.6138 - accuracy: 0.7239 - val_loss: 0.3926 - val_accuracy: 0.8832
Epoch 7/20
512/512 [==============================] - 1s 1ms/step - loss: 0.6059 - accuracy: 0.7284 - val_loss: 0.3856 - val_accuracy: 0.8877
Epoch 

In [57]:
def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
#plot_history(history)